# Variational AutoEncoder (MLP) Pytorch
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/pytorch/examples/blob/master/mnist/main.py
* https://github.com/pytorch/examples/blob/master/vae/main.py
* http://wiseodd.github.io/techblog/2017/01/24/vae-pytorch/
* https://github.com/wiseodd/generative-models
* http://blog.fastforwardlabs.com/2016/08/22/under-the-hood-of-the-variational-autoencoder-in.html
* http://blog.fastforwardlabs.com/2016/08/12/introducing-variational-autoencoders-in-prose-and.html
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
* https://arxiv.org/pdf/1606.05908.pdf
* https://arxiv.org/pdf/1312.6114.pdf

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

use_cuda = torch.cuda.is_available()